In [1]:
# %conda install -c conda-forge ta-lib -q
# %pip install -r requirements.txt -q

# import boto3
import data
import glob
import matplotlib.pyplot as plt
import models
import numpy as np
import os
import pandas as pd
import pypickle
import strategy
import time

from data import * 
from feature_engineering import FeaturesEngineering
from models import Models
from io import StringIO
from IPython.display import display
from sklearn.linear_model import LassoCV
from utils import log

2025-02-24 21:16:37.220674: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-24 21:16:37.231044: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740402997.243242    7495 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740402997.246882    7495 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-24 21:16:37.259637: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
pypickle_overwrite = True

models = Models(use_xgboost_gpu=True)

In [3]:
dir_project = os.getcwd()
dir_models = os.path.join(dir_project, 'models')
dir_models_base = os.path.join(dir_project, 'models', 'base')
dir_models_scaler = os.path.join(dir_project, 'models', 'scaler')
dir_models_best = os.path.join(dir_project, 'models', 'best')

print(f"""
Project directory: {dir_project}
Models directory: {dir_models}
Base models directory: {dir_models_base}
Scalers directory: {dir_models_scaler}
Best models directory: {dir_models_best}
""")


Project directory: /home/alfred/projects/mscfe-masters-project/src
Models directory: /home/alfred/projects/mscfe-masters-project/src/models
Base models directory: /home/alfred/projects/mscfe-masters-project/src/models/base
Scalers directory: /home/alfred/projects/mscfe-masters-project/src/models/scaler
Best models directory: /home/alfred/projects/mscfe-masters-project/src/models/best



# Read raw data

In [4]:
# Define S3 bucket and file path
bucket_name = 'mscfe-capstone-project-bucket'
file_key = 'filtered_data.csv'

filtered = pd.read_csv('filtered_data.csv').drop(['Unnamed: 0'], axis=1)
display(filtered)

,seriesX,seriesY,correlation,spread_adf_stat,spread_adf_pValue,spread_stationary,series_coint_stat,series_coint_pValue,series_cointegrated
0,GOOG,GOOGL,0.999931,-4.971937,0.000025,True,-4.976339,0.000182,True
1,META,DAXEX_ETF,0.888934,-2.917056,0.043400,True,-3.483091,0.033924,True
2,AMZN,BNBUSDT,0.877059,-3.631203,0.005192,True,-3.545100,0.028601,True
3,AMZN,TRXUSDT,0.814577,-3.869803,0.002269,True,-3.387413,0.043768,True
4,SPY_ETF,AVAXUSDT,0.847292,-3.309927,0.014441,True,-3.698512,0.018406,True
5,SPY_ETF,ETHUSDT,0.862483,-3.428205,0.010034,True,-3.737367,0.016394,True
6,EQQQ_ETF,BNBUSDT,0.886060,-3.816404,0.002744,True,-3.670509,0.019987,True
7,BNBUSDT,ADAUSDT,0.901459,-3.279504,0.015820,True,-3.775562,0.014607,True
8,BNBUSDT,BTCUSDT,0.884604,-2.909961,0.044202,True,-3.393160,0.043116,True
9,BNBUSDT,TRXUSDT,0.933562,-3.656380,0.004770,True,-4.003675,0.007095,True


In [5]:
print('********************')
pairs = filtered['seriesX'].iloc[1:] + ' ' + filtered['seriesY'].iloc[1:]
print('Trading pairs:')
for p in pairs:
    print(p)
print('********************')

unique_tickers = pd.concat([filtered['seriesX'].iloc[1:], filtered['seriesY'].iloc[1:]]).drop_duplicates()
print('Unique tickers:')
for ut in unique_tickers:
    print(ut)
print('********************')

# Preview the source data
file_key = 'df_full.csv'
df_full = pd.read_csv('df_full.csv').drop(['Unnamed: 0'], axis=1)
# obj = s3.get_object(Bucket=bucket_name, Key=file_key)
# df_full = pd.read_csv(StringIO(obj['Body'].read().decode('utf-8'))).drop(['Unnamed: 0'], axis=1)
display(df_full.sample(5))

********************
Trading pairs:
META DAXEX_ETF
AMZN BNBUSDT
AMZN TRXUSDT
SPY_ETF AVAXUSDT
SPY_ETF ETHUSDT
EQQQ_ETF BNBUSDT
BNBUSDT ADAUSDT
BNBUSDT BTCUSDT
BNBUSDT TRXUSDT
BNBUSDT XRPUSDT
********************
Unique tickers:
META
AMZN
SPY_ETF
EQQQ_ETF
BNBUSDT
DAXEX_ETF
TRXUSDT
AVAXUSDT
ETHUSDT
ADAUSDT
BTCUSDT
XRPUSDT
********************


,OpenTime,Open,High,Low,Close,Symbol
81718,2025-01-20 07:25:00,43086.0000,43086.000,43086.0000,43086.0000,EQQQ_ETF
392519,2025-01-20 15:45:00,103269.9600,103959.990,103261.6700,103518.8100,BTCUSDT
337565,2025-01-01 09:50:00,2.2521,2.263,2.2514,2.2599,XRPUSDT
340088,2025-01-02 06:55:00,711.5500,712.040,711.1800,711.9000,BNBUSDT
304071,2024-12-20 18:45:00,3486.6500,3489.640,3480.4900,3485.9100,ETHUSDT


## Placeholders

In [6]:
dict_data = {}
dict_cv_features = {}

# Features Engineering

## Create and normalizing features

In [7]:
# Initializing the FeaturesEngineering obj
feat = FeaturesEngineering()

# Default config
feat.default_config

{'ema': [8, 21, 55],
 'macd': {'fast': 12, 'slow': 26, 'signal': 9},
 'rsi': [14],
 'bb': {'timeperiod': 20, 'nbdevup': 2, 'nbdevdn': 2},
 'atr': {'timeperiod': 14},
 'stoch': {'fastk_period': 14, 'slowk_period': 3, 'slowd_period': 3},
 'cci': {'timeperiod': 20},
 'willr': {'timeperiod': 14}}

1. Create features from pairs data
2. Load saved scaler
3. Perform normalization / standardization

In [8]:
scaler_type = "StandardScaler"
dict_data['df_feat'] = {}
dict_data['df_norm'] = {}
dict_data['scaler'] = {}
i = 1

for p in pairs:

    print(f"({i}) Pairs: {p}")
    tickerX, tickerY = p.split(' ')
    
    # 1. Create features from pairs data
    df_feat = feat.create_features(
        tickerX, 
        tickerY,
        df_full[df_full['Symbol']==tickerX],
        df_full[df_full['Symbol']==tickerY],
        config=None, # using default config in the FeaturesEngineering
        dropna=True
    )

    # 2. Load saved scaler
    scaler_name = f"{scaler_type.lower()}_{p.replace(' ', '_')}.pkl"
    dir_scaler = os.path.join(dir_models_scaler, scaler_name)
    scaler = pypickle.load(dir_scaler)

    # 3. Perform normalization / standardization
    data_scaled = scaler.transform(df_feat)
    df_norm = pd.DataFrame(
        data_scaled, index=df_feat.index, columns=df_feat.columns
    )
    df_norm = df_norm.rename(columns={'Spread':'NormalizedSpread'})
    
    dict_data['df_feat'][p] = df_feat
    dict_data['df_norm'][p] = df_norm
    dict_data['scaler'][p] = scaler

    i+=1
    print('')

(1) Pairs: META DAXEX_ETF
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_META_DAXEX_ETF.pkl]

(2) Pairs: AMZN BNBUSDT
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_AMZN_BNBUSDT.pkl]

(3) Pairs: AMZN TRXUSDT
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_AMZN_TRXUSDT.pkl]

(4) Pairs: SPY_ETF AVAXUSDT
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_SPY_ETF_AVAXUSDT.pkl]

(5) Pairs: SPY_ETF ETHUSDT
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_SPY_ETF_ETHUSDT.pkl]

(6) Pairs: EQQQ_ETF BNBUSDT
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_EQQQ_ETF_BNBUSDT.pkl]

(7) Pairs: BNBUSDT ADAUSDT
[pypickle] Pickle file loaded: [/ho

In [9]:
df_norm

,BNBUSDT_EMA8,BNBUSDT_EMA21,BNBUSDT_EMA55,BNBUSDT_MACD,BNBUSDT_RSI14,BNBUSDT_BBupper,BNBUSDT_BBmiddle,BNBUSDT_BBlower,BNBUSDT_ATR14,BNBUSDT_StochK14,...,XRPUSDT_RSI14,XRPUSDT_BBupper,XRPUSDT_BBmiddle,XRPUSDT_BBlower,XRPUSDT_ATR14,XRPUSDT_StochK14,XRPUSDT_StochD3,XRPUSDT_CCI20,XRPUSDT_WILLR14,NormalizedSpread
OpenTime,,,,,,,,,,,,,,,,,,,,,
2024-10-04 20:30:00,-1.680235,-1.686912,-1.672894,0.015952,1.319364,-1.707088,-1.695708,-1.679849,-0.863332,1.416107,...,0.353140,-1.137817,-1.135206,-1.132089,-0.883651,0.970838,0.240717,0.903191,1.344690,-1.665027
2024-10-04 20:35:00,-1.673138,-1.683431,-1.672055,0.133083,1.657784,-1.698144,-1.692503,-1.682527,-0.831344,1.416107,...,0.616383,-1.137577,-1.135090,-1.132100,-0.884681,1.322502,0.896639,1.083912,1.344690,-1.654194
2024-10-04 20:40:00,-1.666828,-1.679943,-1.671119,0.234829,1.761771,-1.690607,-1.688853,-1.682866,-0.857953,1.416107,...,0.616383,-1.137367,-1.134990,-1.132115,-0.892122,1.463168,1.297480,0.964370,1.344690,-1.650581
2024-10-04 20:45:00,-1.664293,-1.677744,-1.670599,0.276419,1.075843,-1.686307,-1.686271,-1.682050,-0.826350,1.234789,...,0.016140,-1.137333,-1.134974,-1.132117,-0.887683,1.240081,1.390479,0.447084,0.729649,-1.661408
2024-10-04 20:50:00,-1.663113,-1.676068,-1.670224,0.293876,0.864854,-1.683056,-1.684223,-1.681239,-0.853316,0.993031,...,-0.429623,-1.137368,-1.134996,-1.132125,-0.885182,0.825776,1.218906,0.040351,0.202471,-1.665011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-20 19:35:00,0.624619,0.608262,0.611732,0.485135,0.950306,0.604907,0.592423,0.578171,-0.169055,1.243066,...,0.245891,1.758357,1.734044,1.708411,1.144744,0.158005,0.212806,0.387378,0.258936,0.623280
2025-01-20 19:40:00,0.630192,0.612101,0.613190,0.554318,1.087021,0.601049,0.600597,0.598644,-0.216222,1.377433,...,0.336222,1.748690,1.739917,1.730243,1.034497,0.261611,0.200114,0.437498,0.633000,0.630279
2025-01-20 19:45:00,0.620045,0.609665,0.612264,0.387205,-0.515124,0.594588,0.603811,0.611756,0.200332,0.418857,...,-1.111384,1.741011,1.741962,1.742272,1.640611,-0.208626,0.072836,-1.391789,-1.466926,0.564991


## Read LassoCV selected features

In [10]:
lasso_cv_name = f"lasso_cv_features.pkl"
dir_lasso = os.path.join(dir_models, lasso_cv_name)
dict_cv_features = pypickle.load(dir_lasso)

for key in dict_cv_features:
    print(f"{key}: {dict_cv_features[key]}")

[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/lasso_cv_features.pkl]
META DAXEX_ETF: ['META_EMA8', 'META_EMA55', 'META_RSI14', 'META_ATR14', 'META_StochD3', 'META_CCI20', 'META_WILLR14', 'DAXEX_ETF_EMA8', 'DAXEX_ETF_EMA21', 'DAXEX_ETF_MACD', 'DAXEX_ETF_BBupper', 'DAXEX_ETF_BBmiddle', 'DAXEX_ETF_BBlower', 'DAXEX_ETF_StochD3']
AMZN BNBUSDT: ['AMZN_EMA8', 'AMZN_EMA21', 'AMZN_EMA55', 'AMZN_RSI14', 'AMZN_ATR14', 'AMZN_StochD3', 'AMZN_WILLR14', 'BNBUSDT_EMA8', 'BNBUSDT_EMA55', 'BNBUSDT_MACD', 'BNBUSDT_RSI14', 'BNBUSDT_ATR14', 'BNBUSDT_StochD3', 'BNBUSDT_CCI20', 'BNBUSDT_WILLR14']
AMZN TRXUSDT: ['AMZN_EMA8', 'AMZN_EMA55', 'AMZN_RSI14', 'AMZN_ATR14', 'AMZN_StochD3', 'AMZN_CCI20', 'AMZN_WILLR14', 'TRXUSDT_MACD']
SPY_ETF AVAXUSDT: ['SPY_ETF_EMA8', 'SPY_ETF_EMA55', 'SPY_ETF_MACD', 'SPY_ETF_RSI14', 'SPY_ETF_BBupper', 'SPY_ETF_BBmiddle', 'SPY_ETF_BBlower', 'SPY_ETF_ATR14', 'SPY_ETF_StochK14', 'SPY_ETF_StochD3', 'SPY_ETF_CCI20', 'SPY_ETF_WILLR14', 'AVAXUSDT_E

# Hyperparameters Tunning

## LSTM Regression

In [12]:
dir_best = os.path.join(dir_models_best, 'lstm')
param_grid = {
    'units': [64, 128, 256],
    'lookback': [10, 5, 15],  # Number of steps to lookback
    'dropout_rate': [0.25],  # Dropout rates
    'batch_size': [32, 64], # Number of data per batch
}

verbose = True
tuned_params = []
i = 1

for p in pairs:
    print(f"({i}) Pairs: {p}")

    # 1. Read normalized features
    df_norm = dict_data['df_norm'][p]
    
    # 2. Create X, y based on LassoCV
    lasso_features_cols = dict_cv_features[p]
    target_col = df_norm.columns.tolist()[-1]
    df_lasso = pd.concat([df_norm[lasso_features_cols], df_norm[target_col]], axis=1)

    # 3. Run Hyperparameters tunning
    start_time = time.perf_counter()
    best_model, best_params, best_mse, best_df_test = models.hyperparameter_tuning(
        df=df_lasso, p=p, model_func=models.rnn_regression, param_grid=param_grid, verbose=verbose)
    end_time = time.perf_counter()
    elapsed_time = end_time - start_time

    stats = {
        'pair':p, 
        'best_mse': best_mse,
        'data_points': len(df_lasso),
        'features': len(df_lasso.columns),
    }
    stats.update(best_params)
    tuned_params.append(stats)

    print(f"Best MSE: {best_mse:.6f}")
    print(f"units: {best_params['units']}, lookback: {best_params['lookback']}, dropout_rate: {best_params['dropout_rate']}, batch_size: {best_params['batch_size']}")
    print(f"Time taken: {elapsed_time:.6f} seconds")

    # 4. Save best model future use like testing and strategy
    model_name = f"best_lstm_{p.replace(' ', '_')}.pkl"
    dir_model_name = os.path.join(dir_best, model_name)
    pypickle.save(dir_model_name, best_model, overwrite=pypickle_overwrite)

    i+=1
    print('')

dir_param = os.path.join(dir_best, 'best_model_results.csv')
df_lstm = pd.DataFrame(tuned_params).set_index('pair')
df_lstm.to_csv(dir_param)
display(df_lstm)

(1) Pairs: META DAXEX_ETF
Testing params: {'units': 64, 'lookback': 10, 'dropout_rate': 0.25, 'batch_size': 32}


I0000 00:00:1740403033.451755    7495 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21964 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9
I0000 00:00:1740403034.972148    7661 service.cc:148] XLA service 0x7bb4d4003160 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1740403034.972278    7661 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2025-02-24 21:17:15.005310: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1740403035.084404    7661 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1740403035.555359    7661 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step 
RNN MSE: 0.027271755008479427
Testing params: {'units': 64, 'lookback': 10, 'dropout_rate': 0.25, 'batch_size': 64}
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step 
RNN MSE: 0.02161120370885995
Testing params: {'units': 64, 'lookback': 5, 'dropout_rate': 0.25, 'batch_size': 32}
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step 
RNN MSE: 0.018882445406812567
Testing params: {'units': 64, 'lookback': 5, 'dropout_rate': 0.25, 'batch_size': 64}
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step 
RNN MSE: 0.030247614755980413
Testing params: {'units': 64, 'lookback': 15, 'dropout_rate': 0.25, 'batch_size': 32}
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step 
RNN MSE: 0.025447826142974982
Testing params: {'units': 64, 'lookback': 15, 'dropout_rate': 0.25, 'batch_size': 64}
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step 
RNN MSE: 0.024239270271585258
Testing params: {'units': 128, 'lookback': 10, 'dropout_rate': 0.25, 'batch_size': 32}
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step 
RNN MSE: 0.04136266586115297
Test

,best_mse,data_points,features,units,lookback,dropout_rate,batch_size
pair,,,,,,,
META DAXEX_ETF,0.018882,1145,15,64,5,0.25,32
AMZN BNBUSDT,0.002946,4991,16,256,10,0.25,32
AMZN TRXUSDT,0.001328,4991,9,256,15,0.25,32
SPY_ETF AVAXUSDT,0.025010,4997,21,256,10,0.25,32
SPY_ETF ETHUSDT,0.001900,4997,16,128,15,0.25,32
EQQQ_ETF BNBUSDT,0.003296,3217,13,256,15,0.25,64
BNBUSDT ADAUSDT,0.000428,31051,9,64,15,0.25,64
BNBUSDT BTCUSDT,0.000305,31051,9,256,5,0.25,32
BNBUSDT TRXUSDT,0.000462,31051,12,64,15,0.25,32
